#### Import

In [4]:
import duckdb
con = duckdb.connect('data.db')

In [5]:
# show table
con.sql('show tables')

┌─────────────────────────────┐
│            name             │
│           varchar           │
├─────────────────────────────┤
│ Bus_Stop_Info               │
│ Business_Operation          │
│ MRT_Business_Area           │
│ MRT_Event                   │
│ MRT_Flow_Record             │
│ MRT_Station_Info            │
│ Representative              │
│ Ubike_Station_Info          │
│ Ubike_Station_Rental_Record │
│ Village_Info                │
│ Village_Population_By_Age   │
│ shop_rental_listing         │
├─────────────────────────────┤
│           12 rows           │
└─────────────────────────────┘

## 查詢周邊商圈、租房資訊

### 商圈平均租金以及周邊捷運站

In [3]:
def get_organization_data(district = None):
    # 檢查使用者是否勾選 district，若有則根據選擇的區域回傳，否則回傳全部
    where_clause = f"WHERE district = '{district}'" if district else ""
    
    res = con.sql(f"""--sql
        WITH nearest_stations AS (
            SELECT 
                s.district,
                s.case_name,
                s.address,
                s.monthly_rent,
                s.area_ping,
                m.station_id,
                m.station_name,
                MIN(
                    6371 * ACOS(
                        COS(RADIANS(s.latitude)) * COS(RADIANS(m.latitude)) *
                        COS(RADIANS(m.longitude) - RADIANS(s.longitude)) +
                        SIN(RADIANS(s.latitude)) * SIN(RADIANS(m.latitude))
                    )
                ) AS nearest_distance_km
            FROM Shop_Rental_Listing s
            CROSS JOIN MRT_Station_Info m
            GROUP BY s.district, s.case_name, s.address, s.monthly_rent, s.area_ping, m.station_id, m.station_name
            HAVING nearest_distance_km <= 1
        )
        SELECT 
            mba.name , -- 商圈名稱
            ROUND(AVG(ns.monthly_rent)) AS average_monthly_rent,
            ns.station_name,
            mba.tag ,
            ns.district
        FROM nearest_stations ns
        JOIN MRT_Business_Area mba ON ns.station_id = mba.station_id -- 加入商圈數據
        {where_clause} -- 動態加入條件
        GROUP BY mba.name, mba.tag, ns.station_name, ns.district
        ORDER BY ns.district, ns.station_name; 
        """)
    
    return res

# 範例
get_organization_data(district = '中山區')
# get_organization_data()

CatalogException: Catalog Error: Table with name Shop_Rental_Listing does not exist!
Did you mean "pg_constraint"?

In [4]:
def get_filtered_shop_rentals(min_rent=None, max_rent=None, min_area=None, max_area=None):
    # 動態構建 WHERE 條件
    conditions = []
    if min_rent is not None:
        conditions.append(f"s.monthly_rent >= {min_rent}")
    if max_rent is not None:
        conditions.append(f"s.monthly_rent <= {max_rent}")
    if min_area is not None:
        conditions.append(f"s.area_ping >= {min_area}")
    if max_area is not None:
        conditions.append(f"s.area_ping <= {max_area}")
    
    # 將條件組合成 WHERE 子句
    where_clause = "WHERE " + " AND ".join(conditions) if conditions else ""
    
    # 執行 SQL 查詢
    query = f"""--sql
    SELECT 
        s.address, 
        s.monthly_rent, 
        ROUND(s.monthly_rent / s.area_ping) AS monthly_rent_per_ping, 
        s.area_ping, 
        s.shop_floor, 
        s.total_floor,
        s.deposit, 
        r.name, 
        r.phone
    FROM Shop_Rental_Listing s
    LEFT JOIN Representative r ON s.phone = r.phone
    {where_clause} -- 動態加入條件
    """
    res = con.sql(query)
    return res

# 範例
get_filtered_shop_rentals(min_rent=10000, max_rent=30000, min_area=10, max_area=50)
# get_filtered_shop_rentals()


┌──────────────────────────┬──────────────┬───────────────────────┬───────────┬────────────┬─────────────┬─────────┬──────────────────────┬────────────┐
│         address          │ monthly_rent │ monthly_rent_per_ping │ area_ping │ shop_floor │ total_floor │ deposit │         name         │   phone    │
│         varchar          │    int32     │        double         │   int32   │  varchar   │    int32    │  int32  │       varchar        │  varchar   │
├──────────────────────────┼──────────────┼───────────────────────┼───────────┼────────────┼─────────────┼─────────┼──────────────────────┼────────────┤
│ 台北市大安區復興南路一段 │        24500 │                2227.0 │        11 │ 1          │          11 │   49000 │ 【格瑞租屋】1-林先生 │ 0223631233 │
│ 台北市大安區四維路       │        20000 │                2000.0 │        10 │ 1          │          19 │   40000 │ 信義敦南加盟營業處   │ 0227056969 │
│ 台北市萬華區寶興街       │        30000 │                2000.0 │        15 │ 1          │           2 │   60000 │ 姜伯平          

## 店面資料與所在商圈及鄰近捷運站

In [7]:
con.sql("""--sql
    WITH distances AS (
    SELECT
        s.case_id,
        s.district,
        s.village,
        s.case_name,
        s.monthly_rent,
        s.area_ping,
        s.address,
        m.station_id,
        m.station_name,
        (
            6371 * ACOS(
                COS(RADIANS(s.latitude)) * COS(RADIANS(m.latitude)) *
                COS(RADIANS(m.longitude) - RADIANS(s.longitude)) +
                SIN(RADIANS(s.latitude)) * SIN(RADIANS(m.latitude))
            )
        ) AS distance_km
    FROM Shop_Rental_Listing s
    CROSS JOIN MRT_Station_Info m
),
nearest_stations AS (
    SELECT
        d.case_id, 
        d.district,
        d.case_name,
        d.address,
        d.village,
        d.station_id,
        d.station_name,
        d.monthly_rent,
        d.area_ping,
        MIN(d.distance_km) AS nearest_distance_km
    FROM distances d
    GROUP BY d.case_id, d.district, d.case_name, d.address, d.village, d.station_id, d.station_name, d.monthly_rent, d.area_ping
    HAVING MIN(d.distance_km) <= 1
)
SELECT 
    ns.case_id,
    ns.district,
    ns.village,
    ns.address,
    ns.case_name,
    ns.station_id,
    ns.station_name,
    ns.monthly_rent,
    ns.area_ping,
    mba.name, -- 商圈名稱
    mba.tag
FROM nearest_stations ns
JOIN MRT_Business_Area mba ON ns.station_id = mba.station_id -- 加入商圈數據d.
ORDER BY ns.case_id, ns.village, ns.nearest_distance_km ASC;
""")

┌─────────┬──────────┬─────────┬──────────────────────────┬──────────────────────────────────────────────┬────────────┬──────────────┬──────────────┬───────────┬──────────────────────┬──────────┐
│ case_id │ district │ village │         address          │                  case_name                   │ station_id │ station_name │ monthly_rent │ area_ping │         name         │   tag    │
│  int64  │ varchar  │ varchar │         varchar          │                   varchar                    │  varchar   │   varchar    │    int32     │   int32   │       varchar        │ varchar  │
├─────────┼──────────┼─────────┼──────────────────────────┼──────────────────────────────────────────────┼────────────┼──────────────┼──────────────┼───────────┼──────────────────────┼──────────┤
│       1 │ 士林區   │ 仁勇里  │ 台北市士林區文林路       │ 士林夜市TOD1+2樓店面                         │ R16        │ 士林         │        84000 │        19 │ 承德路中古汽車(商圈) │ 買在商圈 │
│       1 │ 士林區   │ 仁勇里  │ 台北市士林區文林路       │ 士林夜市TOD1+2樓店

In [228]:
def get_shop_flow_data(case_name=None, business_area_name=None):
    # 動態構建 WHERE 條件
    conditions = []
    if case_name:
        conditions.append(f"cf.case_name = '{case_name}'")
    if business_area_name:
        conditions.append(f"bai.business_area_name = '{business_area_name}'")
    
    where_clause = "WHERE " + " AND ".join(conditions) if conditions else ""
    
    res = con.sql(f"""--sql
    WITH mrt_distances AS (
        SELECT
            s.case_id,
            s.district,
            s.village,
            s.case_name,
            m.station_id AS mrt_station_id,
            (
                6371 * ACOS(
                    COS(RADIANS(s.latitude)) * COS(RADIANS(m.latitude)) *
                    COS(RADIANS(m.longitude) - RADIANS(s.longitude)) +
                    SIN(RADIANS(s.latitude)) * SIN(RADIANS(m.latitude))
                )
            ) AS mrt_distance_km
        FROM Shop_Rental_Listing s
        CROSS JOIN MRT_Station_Info m
    ),
    ubike_distances AS (
        SELECT
            s.case_id,
            s.district,
            s.village,
            s.case_name,
            u.station_id AS ubike_station_id,
            (
                6371 * ACOS(
                    COS(RADIANS(s.latitude)) * COS(RADIANS(u.latitude)) *
                    COS(RADIANS(u.longitude) - RADIANS(s.longitude)) +
                    SIN(RADIANS(s.latitude)) * SIN(RADIANS(u.latitude))
                )
            ) AS ubike_distance_km
        FROM Shop_Rental_Listing s
        CROSS JOIN Ubike_Station_Info u
    ),
    mrt_nearest_stations AS (
        SELECT 
            d.case_id,
            d.district,
            d.case_name,
            d.village,
            d.mrt_station_id
        FROM mrt_distances d
        WHERE d.mrt_distance_km <= 1
    ),
    ubike_nearest_stations AS (
        SELECT 
            d.case_id,
            d.district,
            d.case_name,
            d.village,
            d.ubike_station_id
        FROM ubike_distances d
        WHERE d.ubike_distance_km <= 1
    ),
    -- 不在此處過濾時段，使 MRT flow 保留所有時段
    mrt_flow_data AS (
        SELECT
            mf.station_id AS mrt_station_id, 
            mf.time_period,
            ROUND(AVG(mf.entrance_count + mf.exit_count)) AS avg_mrt_flow
        FROM MRT_Flow_Record mf
        WHERE mf.date >= CURRENT_DATE - INTERVAL '2 years' AND time_period NOT BETWEEN 2 AND 5
        GROUP BY mf.station_id, mf.time_period
    ),
    ubike_flow_data AS (
        SELECT
            uf.station_id AS ubike_station_id,
            uf.time_period,
            ROUND(AVG(uf.rent_count + uf.return_count)) AS avg_ubike_flow
        FROM Ubike_Station_Rental_Record uf
        WHERE uf.date >= CURRENT_DATE - INTERVAL '2 years'
        GROUP BY uf.station_id, uf.time_period
    ),
    -- 將 mrt_nearest_stations 與 mrt_flow_data JOIN，彙整出以 case_id 為單位的 MRT flow 資料
    mrt_case_flow AS (
        SELECT
            mrt.case_id,
            mrt.district,
            mrt.case_name,
            mrt.village,
            mf.time_period,
            AVG(mf.avg_mrt_flow) AS avg_mrt_flow
        FROM mrt_nearest_stations mrt
        JOIN mrt_flow_data mf ON mf.mrt_station_id = mrt.mrt_station_id
        GROUP BY mrt.case_id, mrt.district, mrt.case_name, mrt.village, mf.time_period
    ),
    -- 將 ubike_nearest_stations 與 ubike_flow_data JOIN，彙整出以 case_id 為單位的 Ubike flow 資料
    ubike_case_flow AS (
        SELECT
            ubike.case_id,
            ubike.district,
            ubike.case_name,
            ubike.village,
            uf.time_period,
            AVG(uf.avg_ubike_flow) AS avg_ubike_flow
        FROM ubike_nearest_stations ubike
        JOIN ubike_flow_data uf ON uf.ubike_station_id = ubike.ubike_station_id
        GROUP BY ubike.case_id, ubike.district, ubike.case_name, ubike.village, uf.time_period
    ),
    -- FULL JOIN 將MRT與Ubike的case flow合併，確保沒有MRT資料的時段依然會出現
    combined_flow AS (
        SELECT
            COALESCE(mcf.case_id, ucf.case_id) AS case_id,
            COALESCE(mcf.district, ucf.district) AS district,
            COALESCE(mcf.case_name, ucf.case_name) AS case_name,
            COALESCE(mcf.village, ucf.village) AS village,
            COALESCE(mcf.time_period, ucf.time_period) AS time_period,
            ROUND(COALESCE(mcf.avg_mrt_flow,0) + COALESCE(ucf.avg_ubike_flow,0)) AS avg_total_flow
        FROM mrt_case_flow mcf
        FULL JOIN ubike_case_flow ucf 
            ON mcf.case_id = ucf.case_id
           AND mcf.time_period = ucf.time_period
    ),
    business_area_info AS (
        SELECT 
            ns.case_id,
            mba.name AS business_area_name,
            mba.tag AS business_area_tag
        FROM mrt_nearest_stations ns
        JOIN MRT_Business_Area mba ON ns.mrt_station_id = mba.station_id
        GROUP BY ns.case_id, mba.name, mba.tag
    )
    SELECT 
        cf.case_id,
        cf.district,
        cf.village,
        cf.case_name,
        bai.business_area_name,
        cf.time_period,
        cf.avg_total_flow
    FROM combined_flow cf
    LEFT JOIN business_area_info bai ON cf.case_id = bai.case_id
    {where_clause}
    ORDER BY cf.district, cf.village, cf.case_name, bai.business_area_name, cf.time_period ASC;
    """)

    return res

# 範例呼叫
get_shop_flow_data(case_name='正隆官邸優質店面一樓', business_area_name='永康(商圈)')
# get_shop_flow_data(case_name='正隆官邸優質店面一樓')
# get_shop_flow_data()


┌─────────┬──────────┬─────────┬──────────────────────┬────────────────────┬─────────────┬────────────────┐
│ case_id │ district │ village │      case_name       │ business_area_name │ time_period │ avg_total_flow │
│  int64  │ varchar  │ varchar │       varchar        │      varchar       │    int64    │     double     │
├─────────┼──────────┼─────────┼──────────────────────┼────────────────────┼─────────────┼────────────────┤
│     254 │ 中正區   │ 文北里  │ 正隆官邸優質店面一樓 │ 永康(商圈)         │           0 │          260.0 │
│     254 │ 中正區   │ 文北里  │ 正隆官邸優質店面一樓 │ 永康(商圈)         │           1 │           17.0 │
│     254 │ 中正區   │ 文北里  │ 正隆官邸優質店面一樓 │ 永康(商圈)         │           2 │            3.0 │
│     254 │ 中正區   │ 文北里  │ 正隆官邸優質店面一樓 │ 永康(商圈)         │           3 │            2.0 │
│     254 │ 中正區   │ 文北里  │ 正隆官邸優質店面一樓 │ 永康(商圈)         │           4 │            3.0 │
│     254 │ 中正區   │ 文北里  │ 正隆官邸優質店面一樓 │ 永康(商圈)         │           5 │            3.0 │
│     254 │ 中正區   │ 文北里  │ 正隆官邸優質店面一樓 │ 

## 村裡人口、年齡、性別

In [44]:
def get_village_data(district=None, village=None):
    # 動態構建 WHERE 條件
    conditions = []
    if district:
        conditions.append(f"vi.district = '{district}'")
    if village:
        conditions.append(f"vi.village = '{village}'")
    
    # 合成 WHERE 子句
    where_clause = "WHERE " + " AND ".join(conditions) if conditions else ""
    res = con.sql(f"""--sql
        SELECT vi.district, vi.village, vi.household_count, vi.avg_income,
        ROUND(AVG(vi.avg_income) OVER (PARTITION BY vi.district)) AS nearby_avg_income, vi.median_income,
        ROUND(AVG(vi.median_income) OVER (PARTITION BY vi.district)) AS nearby_median_income,
        ROUND(vi.male_population * 1.0 / SUM(vi.male_population + vi.female_population) OVER (PARTITION BY vi.village), 4) AS male_population_ratio, 
        ROUND(vi.female_population * 1.0 / SUM(vi.male_population + vi.female_population) OVER (PARTITION BY vi.village), 4) AS female_population_ratio, 
        ROUND(v.age_0_9 * 1.0 / SUM(vi.male_population + vi.female_population) OVER (PARTITION BY vi.village), 4) AS avg_0_9_ratio, 
        ROUND(v.age_10_19 * 1.0 / SUM(vi.male_population + vi.female_population) OVER (PARTITION BY vi.village), 4) AS avg_10_19_ratio, 
        ROUND(v.age_20_29 * 1.0 / SUM(vi.male_population + vi.female_population) OVER (PARTITION BY vi.village), 4) AS avg_20_29_ratio, 
        ROUND(v.age_30_64 * 1.0 / SUM(vi.male_population + vi.female_population) OVER (PARTITION BY vi.village), 4) AS avg_30_64_ratio, 
        ROUND(v.age_over_65 * 1.0 / SUM(vi.male_population + vi.female_population) OVER (PARTITION BY vi.village), 4) AS avg_over_65_ratio,
        ROUND(v.age_0_9 * 1.0 / SUM(vi.male_population + vi.female_population) OVER (PARTITION BY vi.district), 4) AS nearby_0_9_ratio, 
        ROUND(v.age_10_19 * 1.0 / SUM(vi.male_population + vi.female_population) OVER (PARTITION BY vi.district), 4) AS nearby_10_19_ratio, 
        ROUND(v.age_20_29 * 1.0 / SUM(vi.male_population + vi.female_population) OVER (PARTITION BY vi.district), 4) AS nearby_20_29_ratio, 
        ROUND(v.age_30_64 * 1.0 / SUM(vi.male_population + vi.female_population) OVER (PARTITION BY vi.district), 4) AS nearby_30_64_ratio, 
        ROUND(v.age_over_65 * 1.0 / SUM(vi.male_population + vi.female_population) OVER (PARTITION BY vi.district), 4) AS nearby_over_65_ratio 
        FROM Village_Info vi
        LEFT JOIN Village_Population_By_Age v ON vi.district = v.district AND vi.village = v.village
        {where_clause}
    """)
    
    return res

# 範例
get_village_data(district = '士林區', village = '承德里')
# get_village_data(district = '士林區')
# get_village_data()

┌──────────┬─────────┬─────────────────┬────────────┬───────────────────┬───────────────┬──────────────────────┬───────────────────────┬─────────────────────────┬───────────────┬─────────────────┬─────────────────┬─────────────────┬───────────────────┬──────────────────┬────────────────────┬────────────────────┬────────────────────┬──────────────────────┐
│ district │ village │ household_count │ avg_income │ nearby_avg_income │ median_income │ nearby_median_income │ male_population_ratio │ female_population_ratio │ avg_0_9_ratio │ avg_10_19_ratio │ avg_20_29_ratio │ avg_30_64_ratio │ avg_over_65_ratio │ nearby_0_9_ratio │ nearby_10_19_ratio │ nearby_20_29_ratio │ nearby_30_64_ratio │ nearby_over_65_ratio │
│ varchar  │ varchar │      int64      │   int64    │      double       │     int64     │        double        │        double         │         double          │    double     │     double      │     double      │     double      │      double       │      double      │       double

## 競爭市場

In [88]:
def get_village_data(district=None, village=None):
    # 動態構建 WHERE 條件
    conditions = []
    if district:
        conditions.append(f"district = '{district}'")
    if village:
        conditions.append(f"village = '{village}'")

    # 合成 WHERE 子句
    where_clause = "WHERE " + " AND ".join(conditions) if conditions else ""
    
    res = con.sql(f"""--sql
            SELECT district, village, business_type, business_sub_type, COUNT(business_name) as shop_cnt, ROUND(AVG(capital)) as avg_capital
            FROM Business_Operation
            {where_clause}
            GROUP BY district, village, business_type, business_sub_type
          """)
    
    return res

get_village_data(district = '大安區', village = '民輝里')
# get_village_data(district = '大安區')
# get_village_data()

┌──────────┬─────────┬────────────────────────┬──────────────────────────────┬──────────┬─────────────┐
│ district │ village │     business_type      │      business_sub_type       │ shop_cnt │ avg_capital │
│ varchar  │ varchar │        varchar         │           varchar            │  int64   │   double    │
├──────────┼─────────┼────────────────────────┼──────────────────────────────┼──────────┼─────────────┤
│ 大安區   │ 民輝里  │ 批發及零售業           │ 布疋及服飾品零售業           │        2 │   1100000.0 │
│ 大安區   │ 民輝里  │ 住宿及餐飲業           │ 餐食業                       │        6 │    126833.0 │
│ 大安區   │ 民輝里  │ 批發及零售業           │ 食品、飲料及菸草製品批發業   │        1 │      3000.0 │
│ 大安區   │ 民輝里  │ 批發及零售業           │ 汽機車及其零配件、用品零售業 │        2 │   1482500.0 │
│ 大安區   │ 民輝里  │ 專業、科學及技術服務業 │ 管理顧問業                   │        2 │     75000.0 │
│ 大安區   │ 民輝里  │ 專業、科學及技術服務業 │ 專門設計業                   │        3 │    516667.0 │
│ 大安區   │ 民輝里  │ 專業、科學及技術服務業 │ 其他專業、科學及技術服務業   │        2 │    225000.0 │
│ 大安區   │ 民輝里  │ 

## 該村里前五個該商圈最多的 business_sub_type 

In [93]:
def get_village_data(district=None, village=None):
    # 動態構建 WHERE 條件
    conditions = []
    if district:
        conditions.append(f"district = '{district}'")
    if village:
        conditions.append(f"village = '{village}'")

    # 合成 WHERE 子句
    where_clause = "WHERE " + " AND ".join(conditions) if conditions else ""
    
    res = con.sql(f"""--sql
            SELECT district, village, business_type, business_sub_type, COUNT(business_name) as shop_cnt, ROUND(AVG(capital)) as avg_capital
            FROM Business_Operation
            {where_clause}
            GROUP BY district, village, business_type, business_sub_type
            ORDER BY shop_cnt DESC
            LIMIT 5
          """)
    
    return res

get_village_data(district = '大安區', village = '民輝里')
# get_village_data(district = '大安區')
# get_village_data()

┌──────────┬─────────┬────────────────────────┬──────────────────────┬──────────┬─────────────┐
│ district │ village │     business_type      │  business_sub_type   │ shop_cnt │ avg_capital │
│ varchar  │ varchar │        varchar         │       varchar        │  int64   │   double    │
├──────────┼─────────┼────────────────────────┼──────────────────────┼──────────┼─────────────┤
│ 大安區   │ 民輝里  │ 住宿及餐飲業           │ 餐食業               │        6 │    126833.0 │
│ 大安區   │ 民輝里  │ 批發及零售業           │ 其他專賣零售業       │        5 │    150000.0 │
│ 大安區   │ 民輝里  │ 其他服務業             │ 美髮及美容美體業     │        4 │    150000.0 │
│ 大安區   │ 民輝里  │ 批發及零售業           │ 家用器具及用品零售業 │        4 │    150750.0 │
│ 大安區   │ 民輝里  │ 專業、科學及技術服務業 │ 廣告業               │        3 │    170000.0 │
└──────────┴─────────┴────────────────────────┴──────────────────────┴──────────┴─────────────┘

## 插入房東資料

In [105]:
def insert_representative(phone, name, is_agent=True):
    # 檢查是否已存在
    existing = con.sql(f"""--sql
    SELECT *
    FROM Representative 
    WHERE phone = '{phone}'
    """).df()

    # 如果不存在，則插入新記錄
    if existing.empty:
        con.sql(f"""--sql
        INSERT INTO Representative (phone, name, is_agent)
        VALUES ('{phone}', '{name}', {is_agent})
        """)
        return f"成功新增房東資料 : {phone}"
    else:
        return f"房東資料已存在"
    
# 範例
insert_representative('0912345678', 'test')

'房東資料已存在'

## 插入Shop_Rental_listing

In [147]:
def insert_shop_rental_listing(case_name, address, longitude, latitude, district, village, monthly_rent, deposit, area_ping, shop_floor,
                               total_floor, phone, is_available=True):
    existing = con.sql(f"""--sql
        SELECT *
        FROM Shop_Rental_Listing 
        WHERE case_name = '{case_name}' 
            AND address = '{address}' 
            AND longitude = '{longitude}' 
            AND latitude = '{latitude}' 
            AND district = '{district}' 
            AND village = '{village}' 
            AND monthly_rent = '{monthly_rent}' 
            AND deposit = '{deposit}' 
            AND shop_floor = '{shop_floor}' 
            AND total_floor = '{total_floor}' 
            AND phone = '{phone}'
    """).df()
    if existing.empty:
        con.sql(f"""--sql
        INSERT INTO Shop_Rental_Listing (
            case_id, case_name, address, longitude, latitude, district,
            village, monthly_rent, deposit, area_ping, shop_floor,
            total_floor, phone, is_available
        )
        VALUES (
            nextval('id_sequence'), '{case_name}', '{address}', {longitude}, {latitude},
            '{district}', '{village}', {monthly_rent}, {deposit}, {area_ping},
            {shop_floor}, {total_floor}, '{phone}', {is_available}
        )
        """)
        return f"成功新增一筆出租資訊"
    else:
        return f"此筆出租資訊已存在"

# 範例
insert_shop_rental_listing('test_case2', 'address', 0, 0, 'district', 'village', 0, 0, 0, '0', 0, '09')

'此筆出租資訊已存在'

In [161]:
con.sql("""--sql
    select *
    from Shop_Rental_Listing
    --where case_name = 'test_case'
""")

┌─────────┬───────────────────────────────────────────────────┬──────────────────────────┬─────────────────┬─────────────────┬──────────┬─────────┬──────────────┬─────────┬───────────┬────────────┬─────────────┬────────────┬──────────────┐
│ case_id │                     case_name                     │         address          │    longitude    │    latitude     │ district │ village │ monthly_rent │ deposit │ area_ping │ shop_floor │ total_floor │   phone    │ is_available │
│  int64  │                      varchar                      │         varchar          │     double      │     double      │ varchar  │ varchar │    int32     │  int32  │   int32   │  varchar   │    int32    │  varchar   │   boolean    │
├─────────┼───────────────────────────────────────────────────┼──────────────────────────┼─────────────────┼─────────────────┼──────────┼─────────┼──────────────┼─────────┼───────────┼────────────┼─────────────┼────────────┼──────────────┤
│       1 │ 士林夜市TOD1+2樓店面               

## 更新狀態為已出租

In [164]:
def mark_as_rented(case_name, address, phone):
    result = con.sql(f"""--sql
    UPDATE Shop_Rental_Listing
    SET is_available = FALSE
    WHERE case_name = '{case_name}' AND address = '{address}'AND phone = '{phone}';
""")  
    
    return f"成功將出租資訊 `{case_name}` 標記為已出租"

mark_as_rented('test_case', 'address', '09')

'成功將出租資訊 `test_case` 標記為已出租'

## 店舖資料

In [172]:
def get_business_data(business_sub_type=None, district=None, village=None):
    # 動態構建 WHERE 條件
    conditions = []
    if business_sub_type:
        conditions.append(f"business_sub_type = '{business_sub_type}'")
    if district:
        conditions.append(f"district = '{district}'")
    if village:
        conditions.append(f"village = '{village}'")

    # 合成 WHERE 子句
    where_clause = "WHERE " + " AND ".join(conditions) if conditions else ""
    res = con.sql(f"""--sql
            SELECT business_name, address, capital, longitude, latitude, district, village
            FROM Business_Operation 
            {where_clause}
          """)
    
    return res

# 範例
get_business_data('布疋及服飾品零售業')

┌──────────────────┬─────────────────────────────────────────────┬──────────┬────────────┬───────────┐
│  business_name   │                   address                   │ capital  │ longitude  │ latitude  │
│     varchar      │                   varchar                   │  double  │   double   │  double   │
├──────────────────┼─────────────────────────────────────────────┼──────────┼────────────┼───────────┤
│ 季候風服飾店     │ 臺北市信義區永吉里11鄰永吉路517巷8弄4號     │ 100000.0 │ 121.580284 │ 25.046455 │
│ 沛翔飾品店       │ 臺北市大安區建安里14鄰敦化南路一段187巷15號 │ 100000.0 │ 121.550026 │ 25.042667 │
│ 珍珠服飾商行     │ 臺北市信義區永吉里8鄰永吉路443巷30號        │ 100000.0 │ 121.578322 │ 25.046282 │
│ 日曜陽選物工作室 │ 臺北市大同區建泰里8鄰承德路一段41巷16號     │  50000.0 │ 121.517729 │ 25.051155 │
│ 鍚光普照商行     │ 臺北市中山區恆安里1鄰中山北路三段15之1號    │ 200000.0 │ 121.522527 │ 25.064273 │
│ 喜樂服飾行       │ 臺北市中正區光復里7鄰武昌街一段22巷8號      │  10000.0 │  121.51222 │ 25.043799 │
│ 金能服飾行       │ 臺北市萬華區萬壽里6鄰西寧南路36號           │  90000.0 │  121.50664 │ 25.045591 │
│ 柏毓行           │ 臺北市萬華區

In [12]:
def get_organization_flow_data(avg_total_flow=None, time_period=None):
    # 動態構建 WHERE 條件
    conditions = []
    if avg_total_flow is not None:
        conditions.append(f"cf.avg_total_flow >= {avg_total_flow}")
    if time_period is not None:
        conditions.append(f"cf.time_period = {time_period}")
    
    where_clause = "WHERE " + " AND ".join(conditions) if conditions else ""
    
    res = con.sql(f"""--sql
    WITH mrt_distances AS (
        SELECT
            s.case_id,
            s.district,
            s.village,
            s.case_name,
            m.station_id AS mrt_station_id,
            (
                6371 * ACOS(
                    COS(RADIANS(s.latitude)) * COS(RADIANS(m.latitude)) *
                    COS(RADIANS(m.longitude) - RADIANS(s.longitude)) +
                    SIN(RADIANS(s.latitude)) * SIN(RADIANS(m.latitude))
                )
            ) AS mrt_distance_km
        FROM Shop_Rental_Listing s
        CROSS JOIN MRT_Station_Info m
    ),
    ubike_distances AS (
        SELECT
            s.case_id,
            s.district,
            s.village,
            s.case_name,
            u.station_id AS ubike_station_id,
            (
                6371 * ACOS(
                    COS(RADIANS(s.latitude)) * COS(RADIANS(u.latitude)) *
                    COS(RADIANS(u.longitude) - RADIANS(s.longitude)) +
                    SIN(RADIANS(s.latitude)) * SIN(RADIANS(u.latitude))
                )
            ) AS ubike_distance_km
        FROM Shop_Rental_Listing s
        CROSS JOIN Ubike_Station_Info u
    ),
    -- 只取最近的捷運站
    mrt_nearest_stations AS (
        SELECT case_id, district, case_name, village, mrt_station_id FROM (
            SELECT d.*,
                   ROW_NUMBER() OVER(PARTITION BY d.case_id ORDER BY d.mrt_distance_km) AS rn
            FROM mrt_distances d
            WHERE d.mrt_distance_km <= 1
        ) temp
        WHERE rn = 1
    ),
    ubike_nearest_stations AS (
        SELECT 
            d.case_id,
            d.district,
            d.case_name,
            d.village,
            d.ubike_station_id
        FROM ubike_distances d
        WHERE d.ubike_distance_km <= 1
    ),
    mrt_flow_data AS (
        SELECT
            mf.station_id AS mrt_station_id, 
            mf.time_period,
            ROUND(AVG(mf.entrance_count + mf.exit_count)) AS avg_mrt_flow
        FROM MRT_Flow_Record mf
        WHERE mf.date >= CURRENT_DATE - INTERVAL '2 years' 
          AND mf.time_period NOT BETWEEN 2 AND 5
        GROUP BY mf.station_id, mf.time_period
    ),
    ubike_flow_data AS (
        SELECT
            uf.station_id AS ubike_station_id,
            uf.time_period,
            ROUND(AVG(uf.rent_count + uf.return_count)) AS avg_ubike_flow
        FROM Ubike_Station_Rental_Record uf
        WHERE uf.date >= CURRENT_DATE - INTERVAL '2 years'
        GROUP BY uf.station_id, uf.time_period
    ),
    mrt_case_flow AS (
        SELECT
            mrt.case_id,
            mrt.district,
            mrt.case_name,
            mrt.village,
            m.station_name AS mrt_station_name,
            mf.time_period,
            AVG(mf.avg_mrt_flow) AS avg_mrt_flow
        FROM mrt_nearest_stations mrt
        JOIN mrt_flow_data mf ON mf.mrt_station_id = mrt.mrt_station_id
        JOIN MRT_Station_Info m ON m.station_id = mrt.mrt_station_id
        GROUP BY mrt.case_id, mrt.district, mrt.case_name, mrt.village, m.station_name, mf.time_period
    ),
    ubike_case_flow AS (
        SELECT
            ubike.case_id,
            ubike.district,
            ubike.case_name,
            ubike.village,
            uf.time_period,
            AVG(uf.avg_ubike_flow) AS avg_ubike_flow
        FROM ubike_nearest_stations ubike
        JOIN ubike_flow_data uf ON uf.ubike_station_id = ubike.ubike_station_id
        GROUP BY ubike.case_id, ubike.district, ubike.case_name, ubike.village, uf.time_period
    ),
    combined_flow AS (
        SELECT
            COALESCE(mcf.case_id, ucf.case_id) AS case_id,
            COALESCE(mcf.district, ucf.district) AS district,
            COALESCE(mcf.case_name, ucf.case_name) AS case_name,
            COALESCE(mcf.village, ucf.village) AS village,
            mcf.mrt_station_name,
            COALESCE(mcf.time_period, ucf.time_period) AS time_period,
            ROUND(COALESCE(mcf.avg_mrt_flow,0) + COALESCE(ucf.avg_ubike_flow,0)) AS avg_total_flow
        FROM mrt_case_flow mcf
        FULL JOIN ubike_case_flow ucf 
            ON mcf.case_id = ucf.case_id
           AND mcf.time_period = ucf.time_period
    ),
    business_area_info AS (
        SELECT 
            ns.case_id,
            mba.name AS business_area_name,
            mba.tag AS business_area_tag
        FROM mrt_nearest_stations ns
        JOIN MRT_Business_Area mba ON ns.mrt_station_id = mba.station_id
        GROUP BY ns.case_id, mba.name, mba.tag
    )
    SELECT 
        cf.case_id,
        cf.district AS "區域",
        cf.village AS "村裡",
        bai.business_area_name AS "商圈名稱",
        cf.mrt_station_name AS "捷運站",
        cf.time_period AS "時段",
        cf.avg_total_flow AS "人流"
    FROM combined_flow cf
    LEFT JOIN business_area_info bai ON cf.case_id = bai.case_id
    {where_clause}
    ORDER BY cf.district, cf.village, cf.case_name, bai.business_area_name, cf.mrt_station_name, cf.time_period ASC;
    """)

    return res

# 範例呼叫：過濾人流為100及時段為10的紀錄
get_organization_flow_data(avg_total_flow=2000)


┌─────────┬─────────┬─────────┬────────────────┬──────────┬───────┬────────┐
│ case_id │  區域   │  村裡   │    商圈名稱    │  捷運站  │ 時段  │  人流  │
│  int64  │ varchar │ varchar │    varchar     │ varchar  │ int64 │ double │
├─────────┼─────────┼─────────┼────────────────┼──────────┼───────┼────────┤
│     523 │ 中山區  │ 中原里  │ 四平陽光(商圈) │ 松江南京 │     7 │ 3628.0 │
│     523 │ 中山區  │ 中原里  │ 四平陽光(商圈) │ 松江南京 │     8 │ 9253.0 │
│     523 │ 中山區  │ 中原里  │ 四平陽光(商圈) │ 松江南京 │     9 │ 3856.0 │
│     523 │ 中山區  │ 中原里  │ 四平陽光(商圈) │ 松江南京 │    10 │ 2299.0 │
│     523 │ 中山區  │ 中原里  │ 四平陽光(商圈) │ 松江南京 │    11 │ 2685.0 │
│     523 │ 中山區  │ 中原里  │ 四平陽光(商圈) │ 松江南京 │    12 │ 2840.0 │
│     523 │ 中山區  │ 中原里  │ 四平陽光(商圈) │ 松江南京 │    13 │ 2897.0 │
│     523 │ 中山區  │ 中原里  │ 四平陽光(商圈) │ 松江南京 │    14 │ 2849.0 │
│     523 │ 中山區  │ 中原里  │ 四平陽光(商圈) │ 松江南京 │    15 │ 2686.0 │
│     523 │ 中山區  │ 中原里  │ 四平陽光(商圈) │ 松江南京 │    16 │ 3290.0 │
│       · │   ·     │   ·     │       ·        │   ·      │     · │    ·   │
│       · │   ·     

In [17]:
def get_organization_flow_data(flow_rank=None, time_periods=None):
    # 動態構建 WHERE 條件
    conditions = []
    if flow_rank is not None:
        conditions.append(f"bar.flow_rank >= {flow_rank}")
    if time_periods:
        # 將 time_periods 格式化為 SQL 可接受的 IN 條件
        formatted_periods = ", ".join([f"'{tp}'" for tp in time_periods])
        conditions.append(f"cf.time_group IN ({formatted_periods})")
    
    where_clause = "WHERE " + " AND ".join(conditions) if conditions else ""

    res = con.sql(f"""--sql
    WITH mrt_distances AS (
    SELECT
        s.case_id,
        s.district,
        s.village,
        s.case_name,
        m.station_id AS mrt_station_id,
        (
            6371 * ACOS(
                COS(RADIANS(s.latitude)) * COS(RADIANS(m.latitude)) *
                COS(RADIANS(m.longitude) - RADIANS(s.longitude)) +
                SIN(RADIANS(s.latitude)) * SIN(RADIANS(m.latitude))
            )
        ) AS mrt_distance_km
    FROM Shop_Rental_Listing s
    CROSS JOIN MRT_Station_Info m
    ),
    ubike_distances AS (
        SELECT
            s.case_id,
            s.district,
            s.village,
            s.case_name,
            u.station_id AS ubike_station_id,
            (
                6371 * ACOS(
                    COS(RADIANS(s.latitude)) * COS(RADIANS(u.latitude)) *
                    COS(RADIANS(u.longitude) - RADIANS(s.longitude)) +
                    SIN(RADIANS(s.latitude)) * SIN(RADIANS(u.latitude))
                )
            ) AS ubike_distance_km
        FROM Shop_Rental_Listing s
        CROSS JOIN Ubike_Station_Info u
    ),
    mrt_flow_data AS (
        SELECT
            mf.station_id AS mrt_station_id, 
            FLOOR((mf.time_period - 1) / 2) + 1 AS time_group,
            ROUND(SUM(mf.entrance_count + mf.exit_count)) AS avg_mrt_flow
        FROM MRT_Flow_Record mf
        WHERE mf.date >= CURRENT_DATE - INTERVAL '2 years' 
        AND mf.time_period NOT BETWEEN 2 AND 5
        GROUP BY mf.station_id, time_group
    ),
    ubike_flow_data AS (
        SELECT
            uf.station_id AS ubike_station_id,
            FLOOR((uf.time_period - 1) / 2) + 1 AS time_group,
            ROUND(SUM(uf.rent_count + uf.return_count)) AS avg_ubike_flow
        FROM Ubike_Station_Rental_Record uf
        WHERE uf.date >= CURRENT_DATE - INTERVAL '2 years'
        GROUP BY uf.station_id, time_group
    ),
    mrt_case_flow AS (
        SELECT
            mrt.case_id,
            mrt.district,
            mrt.case_name,
            mrt.village,
            m.station_name AS mrt_station_name,
            mf.time_group,
            SUM(mf.avg_mrt_flow) AS total_mrt_flow
        FROM mrt_distances mrt
        JOIN mrt_flow_data mf ON mf.mrt_station_id = mrt.mrt_station_id
        JOIN MRT_Station_Info m ON m.station_id = mrt.mrt_station_id
        WHERE mrt.mrt_distance_km <= 1
        GROUP BY mrt.case_id, mrt.district, mrt.case_name, mrt.village, m.station_name, mf.time_group
    ),
    ubike_case_flow AS (
        SELECT
            ubike.case_id,
            ubike.district,
            ubike.case_name,
            ubike.village,
            uf.time_group,
            SUM(uf.avg_ubike_flow) AS total_ubike_flow
        FROM ubike_distances ubike
        JOIN ubike_flow_data uf ON uf.ubike_station_id = ubike.ubike_station_id
        WHERE ubike.ubike_distance_km <= 1
        GROUP BY ubike.case_id, ubike.district, ubike.case_name, ubike.village, uf.time_group
    ),
    combined_flow AS (
        SELECT
            COALESCE(mcf.case_id, ucf.case_id) AS case_id,
            COALESCE(mcf.district, ucf.district) AS district,
            COALESCE(mcf.case_name, ucf.case_name) AS case_name,
            COALESCE(mcf.village, ucf.village) AS village,
            mcf.mrt_station_name,
            COALESCE(mcf.time_group, ucf.time_group) AS time_group,
            COALESCE(mcf.total_mrt_flow, 0) + COALESCE(ucf.total_ubike_flow, 0) AS avg_total_flow
        FROM mrt_case_flow mcf
        FULL JOIN ubike_case_flow ucf 
            ON mcf.case_id = ucf.case_id AND mcf.time_group = ucf.time_group
    ),
    business_area_info AS (
        SELECT 
            ns.case_id,
            mba.name AS business_area_name,
            mba.tag AS business_area_tag
        FROM mrt_distances ns
        JOIN MRT_Business_Area mba ON ns.mrt_station_id = mba.station_id
        WHERE ns.mrt_distance_km <= 1
        GROUP BY ns.case_id, mba.name, mba.tag
    ),
    business_area_flow AS (
        SELECT
            bai.business_area_name,
            SUM(cf.avg_total_flow) AS total_flow
        FROM combined_flow cf
        JOIN business_area_info bai ON cf.case_id = bai.case_id
        GROUP BY bai.business_area_name
    ),
    business_area_ranks AS (
        SELECT
            business_area_name,
            total_flow,
            NTILE(10) OVER (ORDER BY total_flow) AS flow_rank -- 分成10個分位數
        FROM business_area_flow
    )
    SELECT 
        cf.case_id,
        cf.district AS district,
        cf.village AS village,
        bai.business_area_name AS business_area_name,
        cf.mrt_station_name AS mrt_station,
        cf.time_group AS time_period,
        cf.avg_total_flow AS flow,
        bar.flow_rank AS flow_rank
    FROM combined_flow cf
    LEFT JOIN business_area_info bai ON cf.case_id = bai.case_id
    LEFT JOIN business_area_ranks bar ON bai.business_area_name = bar.business_area_name
    {where_clause}
    ORDER BY cf.district, cf.village, cf.case_name, bai.business_area_name, cf.mrt_station_name, cf.time_group ASC;
    """)

    df = res.df()
    return df

get_organization_flow_data(flow_rank=2, time_periods=[3,4,5])

,case_id,district,village,business_area_name,mrt_station,time_period,flow,flow_rank
0,523,中山區,中原里,中山北路婚紗(商圈),中山,3.0,717910.0,10
1,523,中山區,中原里,中山北路婚紗(商圈),中山,4.0,8600717.0,10
2,523,中山區,中原里,中山北路婚紗(商圈),中山,5.0,9477381.0,10
3,523,中山區,中原里,中山北路婚紗(商圈),松江南京,3.0,943580.0,10
4,523,中山區,中原里,中山北路婚紗(商圈),松江南京,4.0,17883429.0,10
...,...,...,...,...,...,...,...,...
18427,60,萬華區,頂碩里,萬華街區(商圈),小南門,4.0,2454072.0,5
18428,60,萬華區,頂碩里,萬華街區(商圈),小南門,5.0,1438854.0,5
18429,60,萬華區,頂碩里,萬華街區(商圈),龍山寺,3.0,619109.0,5
18430,60,萬華區,頂碩里,萬華街區(商圈),龍山寺,4.0,4193236.0,5


: 